# Find branch years

### for experiments where I have made a global annual average

In [120]:
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import xarray as xr
import intake

In [52]:
# check which folders are available, and make a set of available experiments

experiments = {}

directory = '../Processed_data/Global_annual_means/'

model_names = [ f.name for f in os.scandir(directory) if f.is_dir() and f.name !='.ipynb_checkpoints']

for model in model_names:
    #print(model)
    experiments[model] = {}
    modeldirectory = os.path.join(directory, model)
    modelexp_names = [ f.name for f in os.scandir(modeldirectory) if f.is_dir() and f.name !='.ipynb_checkpoints']
    for exp in modelexp_names:
        #print(exp)
        experiments[model][exp] = {}
        modelexpdirectory = os.path.join(modeldirectory, exp)
        filenames = [f.name for f in os.scandir(modelexpdirectory) if f.name !='.ipynb_checkpoints']
        
        members = [file.rsplit('_')[2] for file in filenames]
        members.sort()
        experiments[model][exp] = members

experiments   

In [162]:
col_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(col_url)

filename = '../Processed_data/branch_times.txt'
column_names = ['model', 'exp', 'member', 'parent_experiment_id', 'parent_variant_id', 
                'branch_time_in_child', 'branch_time_in_parent', 'parent_time_units', 'branch_method']
df = pd.DataFrame(columns = column_names)

for model in experiments:
    for exp in experiments[model]:
            for member in experiments[model][exp]:

                cat = col.search(experiment_id = exp, variable_id='tas', table_id='Amon', source_id = model, member_id = member)
                dset_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True}, cdf_kwargs={'chunks': {}})
                for key in dset_dict.keys():
                    ds = dset_dict[key]
                parent_experiment_id = ds.parent_experiment_id
                parent_variant_label = ds.parent_variant_label
                branch_time_in_child = ds.branch_time_in_child
                branch_time_in_parent = ds.branch_time_in_parent
                parent_time_units = ds.parent_time_units
                branch_method = ds.branch_method

                print(model, exp, member, parent_experiment_id, parent_variant_label, branch_time_in_child, branch_time_in_parent, parent_time_units, branch_method)
                df_row = pd.DataFrame([[model, exp, member, parent_experiment_id, parent_variant_label,
                           branch_time_in_child, branch_time_in_parent, parent_time_units, branch_method]] ,columns = column_names)
                df = df.append(df_row, ignore_index = True)

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 1 group(s)
AWI-CM-1-1-MR abrupt-4xCO2 r1i1p1f1 piControl r1i1p1f1 0.0 90945.0 days since 2401-01-01 00:00:00
--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 1 group(s)
AWI-CM-1-1-MR piControl r1i1p1f1 piControl-spinup r1i1p1f1 0.0 0.0 days since 1901-01-01 00:00:00
--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 1 group(s)
AWI-CM-1-1-MR ssp585 r1i1p1f1 historical r1i1p1f1 0.0 60265.0 days since 1850-01-01 00:00:00
--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> 

In [163]:
df

,model,exp,member,parent_experiment_id,parent_variant_id,branch_time_in_child,branch_time_in_parent,parent_time_units,branch_method
0,AWI-CM-1-1-MR,abrupt-4xCO2,r1i1p1f1,piControl,r1i1p1f1,0.0,90945.0,days since 2401-01-01 00:00:00,standard
1,AWI-CM-1-1-MR,piControl,r1i1p1f1,piControl-spinup,r1i1p1f1,0.0,0.0,days since 1901-01-01 00:00:00,standard
2,AWI-CM-1-1-MR,ssp585,r1i1p1f1,historical,r1i1p1f1,0.0,60265.0,days since 1850-01-01 00:00:00,standard
3,AWI-CM-1-1-MR,ssp370,r1i1p1f1,historical,r1i1p1f1,0.0,60265.0,days since 1850-01-01 00:00:00,standard
4,AWI-CM-1-1-MR,ssp370,r2i1p1f1,historical,r2i1p1f1,0.0,60265.0,days since 1850-01-01 00:00:00,standard
...,...,...,...,...,...,...,...,...,...
289,BCC-CSM2-MR,historical,r3i1p1f1,piControl,r3i1p1f1,0.0,2439.0,days since 1850-01-01,Standard
290,BCC-CSM2-MR,piControl,r1i1p1f1,piControl-spinup,r1i1p1f1,0.0,0.0,days since 1850-01-01,standard
291,BCC-CSM2-MR,ssp585,r1i1p1f1,historical,r1i1p1f1,0.0,2015.0,days since 2015-01-01,Standard
292,BCC-CSM2-MR,ssp370,r1i1p1f1,historical,r1i1p1f1,0.0,2015.0,days since 2015-01-01,Standard


In [164]:
df.to_csv(filename, ' ')